<a href="https://colab.research.google.com/github/ruanvirginio/scriptsMestrado/blob/main/Multivariado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore")

url_base = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Medicoes_2018-2024.csv"
df = pd.read_csv(url_base,  sep=',', encoding='latin-1', skiprows=1)

url_GD = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/EntrantesGD.csv"
gd = pd.read_csv(url_GD,  sep=';', encoding='latin-1')

url_clientes = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Base_Quantidade_Clientes.csv"
clientes = pd.read_csv(url_clientes, sep=';', encoding='latin-1')

url_clima_patos = "https://media.githubusercontent.com/media/ruanvirginio/scriptsMestrado/refs/heads/main/Clima/dadosSousa.csv"
clima_patos = pd.read_csv(url_clima_patos, sep=';', encoding='latin-1')

#### Funções para tratamento de outliers

In [ ]:
# Função para aplicar o filtro IQR
def filtrar_coluna_iqr(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 3.2 * IQR

    return df[(df[coluna] >= lower_bound) & (df[coluna] <= upper_bound)]

# Função para aplicar o filtro baseado em Média Móvel
def filtrar_coluna_media_movel(df, coluna, janela=20, threshold=4):

    media_movel = df[coluna].rolling(window=janela, center=True).mean()
    desvio = np.abs(df[coluna] - media_movel)

    limite_superior = media_movel + threshold * desvio.std()
    limite_inferior = media_movel - threshold * desvio.std()

    return df[(df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior)]

# Função para aplicar o filtro baseado em Z-Score
def filtrar_coluna_zscore(df, coluna, threshold=3):
    media = df[coluna].mean()
    desvio_padrao = df[coluna].std()
    z_scores = (df[coluna] - media) / desvio_padrao

    return df[np.abs(z_scores) < threshold]

# Função que aplica o filtro escolhido
def aplicar_filtro(df, coluna, metodo='iqr', janela=20, threshold=4, z_threshold=3):
    if metodo == 'iqr':
        return filtrar_coluna_iqr(df, coluna)
    elif metodo == 'media_movel':
        return filtrar_coluna_media_movel(df, coluna, janela=janela, threshold=threshold)
    elif metodo == 'zscore':
        return filtrar_coluna_zscore(df, coluna, threshold=z_threshold)
    else:
        raise ValueError("Método inválido! Escolha entre 'iqr', 'media_movel' ou 'zscore'.")


#### Tratando base de potências

In [ ]:
df['Potência Ativa'] = pd.to_numeric(df['Potência Ativa'].str.replace(',', '.'), errors='coerce')
df['Potência Reativa'] = pd.to_numeric(df['Potência Reativa'].str.replace(',', '.'), errors='coerce')

df.rename(columns={
    'Potência Ativa': 'P',
    'Potência Reativa': 'Q',
    'Data/Hora Medição': 'datahora',
    'Equipamento Medição': 'TRAFO'
}, inplace=True)

# df['datahora'] = pd.to_datetime(df['datahora'], dayfirst=True, errors='coerce')
df['datahora'] = pd.to_datetime(df['datahora'], format='%d/%m/%Y %H:%M:%S')

df['P'] = df['P'].abs()
df['Q'] = df['Q'].abs()
df['S'] = np.sqrt(df['P']**2 + df['Q']**2)  # Calculando a potência aparente (S), em kVA

df = df[df['S'] != 0]

df_filtrado = pd.DataFrame()

metodo_filtro = 'iqr'

for trafo in df['TRAFO'].unique():
    df_trafo = df[df['TRAFO'] == trafo]

    df_trafo_filtrado = aplicar_filtro(df_trafo, 'S', metodo=metodo_filtro, janela=20, threshold=4, z_threshold=3)

    df_filtrado = pd.concat([df_filtrado, df_trafo_filtrado], ignore_index=True)

df_filtrado = df_filtrado.sort_values(by=['TRAFO', 'datahora'])
df_filtrado = df_filtrado.drop_duplicates(subset=['datahora', 'TRAFO'])

df_daily = df_filtrado.groupby('TRAFO', group_keys=False).apply(lambda x: x.set_index('datahora').resample('D').max()).reset_index()

df_count = df_daily.groupby('TRAFO').count()
df_count = df_count.sort_values('datahora').tail(41).reset_index() # esses são os trafos com mais de 97,5% de linhas preenchidas

lista_trafos = df_count['TRAFO'].unique().tolist()

# trafos_escolhidos = lista_trafos

trafos_escolhidos = ['CPX_DJ_12B1', 'LCN_DJ_12B1', 'SPE_DJ_12B2', 'ARA_DJ_12B1', 'ARR_DJ_12B1', 'BQR_DJ_12B1', 'PLT_DJ_12B1', 'PTS_DJ_12B1', 'TXR_DJ_12B1']

df_filtrado = df_daily[df_daily['TRAFO'].isin(lista_trafos)]

data_inicio = '2018-01-01'
data_fim = '2024-12-31'
datas_completas = pd.date_range(start=data_inicio, end=data_fim, freq='D')

def preencher_com_7_linhas_anteriores(grupo):
    grupo = grupo.set_index('datahora')
    grupo = grupo.reindex(datas_completas)

    grupo = grupo.fillna(grupo.shift(7))  # 'shift(7)' pega o valor de 7 linhas antes

    grupo['TRAFO'] = grupo['TRAFO'].iloc[0]  # Reatribui o trafo
    return grupo.reset_index().rename(columns={'index': 'datahora'})

def preencher_com_1_linhas_anteriores(grupo):
    grupo = grupo.set_index('datahora')
    grupo = grupo.reindex(datas_completas)

    grupo = grupo.fillna(grupo.shift(1))  # 'shift(1)' pega o valor de 1 linha antes

    grupo['TRAFO'] = grupo['TRAFO'].iloc[0]
    return grupo.reset_index().rename(columns={'index': 'datahora'})

# Aplicar a função para cada transformador
df_preenchido = df_filtrado.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_7_linhas_anteriores)
df_preenchido = df_preenchido.groupby('TRAFO', group_keys=False).apply(preencher_com_1_linhas_anteriores)


In [ ]:
# df_count2 = df_daily.groupby('TRAFO').count().sort_values('datahora')
# df_count2.tail(50)

#### Tratando dados de GD

In [ ]:
gd.rename(columns={
    'Tempo': 'datahora',
}, inplace=True)

gd['datahora'] = pd.to_datetime(gd['datahora'], format='%Y-%m-%d %H:%M:%S')

trafos = gd["TRAFO"].unique()
trafo_datas = pd.DataFrame([(d, t) for d in datas_completas for t in trafos], columns=["datahora", "TRAFO"])
gd_max = gd.groupby(["datahora", "TRAFO"])['PotenciaAcumulada'].max().reset_index()
gd = trafo_datas.merge(gd_max, on=["datahora", "TRAFO"], how="left").fillna(0)

#### Tratando base de clientes

In [ ]:
clientes['DATA'] = pd.to_datetime(clientes['DATA'])

# One-hot encoding
dummies = pd.get_dummies(clientes[['CLASSE', 'DSC_GRUPO_FORNECIMENTO']].astype(str),
                         prefix=['classe', 'fornec'])

# Multiplicando as colunas pelo valor de QTD_CLIENTES, pra ser um "peso"
dummies_mult = dummies.multiply(clientes['QTD_CLIENTES'], axis=0)

clientes_concat = pd.concat([clientes[['TRAFO', 'DATA']], dummies_mult], axis=1)
clientes2 = clientes_concat.groupby(['TRAFO', 'DATA']).sum().reset_index()
clientes2.drop('classe_0', axis=1) # drop classe que n existe

clientes2.rename(columns={
    'DATA': 'datahora',
}, inplace=True)

In [ ]:
df = pd.merge(df_preenchido, gd, on=['datahora', 'TRAFO'], how='left')

df = pd.merge(df, clientes2, on=['datahora', 'TRAFO'], how='left')

data = df.fillna(0)

#### Algoritmo de Aprendizado de Máquina

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import random

# Fixando seeds para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

def plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo):
    plt.figure(figsize=(14, 6))
    plt.plot(df_previsoes, y_test_inverso, label='Valores Reais', color='blue')
    plt.plot(df_previsoes, y_pred, label=f'Predictions {modelo} - Transformer {trafo}', linestyle='--', color='orange')
    plt.xlabel('Date')
    plt.ylabel('Apparent Power')
    plt.title(f'Predicted x Real Comparison - Transformer {trafo} ({modelo})')
    plt.legend()
    plt.show()

def treinar_e_prever_modelo(data, trafos_escolhidos, modelo, janela, features, epochs=20, batch_size=32):
    resultados = []

    for trafo in trafos_escolhidos:
        df = data[data['TRAFO'] == trafo]
        df = df[['datahora'] + features]  # Mantém somente as colunas necessárias
        df = df.set_index(['datahora'])
        df.sort_index(inplace=True)

        # Normalização das features
        scaler = MinMaxScaler(feature_range=(0, 1))
        dados_normalizados = scaler.fit_transform(df)

        # Separando X (features) e y (variável alvo 'S')
        X, y = [], []
        target_idx = features.index('S')  # Posição de 'S' nas features

        for i in range(janela, len(dados_normalizados)):
            X.append(dados_normalizados[i-janela:i, :])  # Todas as features
            y.append(dados_normalizados[i, target_idx])  # Apenas S

        X, y = np.array(X), np.array(y)

        # Ajuste para LSTM
        if modelo == 'LSTM':
            X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

        # Divisão treino-teste (80/20)
        split = int(len(X) * 0.83)
        X_train, X_test = X[:split], X[split:]
        y_train, y_test = y[:split], y[split:]

        # Definição dos modelos
        if modelo == 'SVR':
            regressor = SVR(kernel='rbf', C=100, gamma=0.001, epsilon=0.01)
        elif modelo == 'GBR':
            regressor = GradientBoostingRegressor(n_estimators=100, random_state=42)
        elif modelo == 'LGBM':
            regressor = LGBMRegressor(n_estimators=100, random_state=42)
        elif modelo == 'XGB':
            regressor = XGBRegressor(n_estimators=100, random_state=42)
        elif modelo == 'CatBoost':
            regressor = CatBoostRegressor(n_estimators=100, random_state=42, verbose=0)
        elif modelo == 'RFR':
            regressor = RandomForestRegressor(n_estimators=100, max_depth=20, max_features='sqrt', n_jobs=-1, random_state=42)
        elif modelo == 'LSTM':
            regressor = Sequential()
            regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
            regressor.add(LSTM(units=50))
            regressor.add(Dense(1))
            regressor.compile(optimizer='adam', loss='mean_squared_error')
            regressor.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Treinamento do modelo (exceto LSTM, que já treinou)
        if modelo != 'LSTM':
            regressor.fit(X_train.reshape(X_train.shape[0], -1), y_train)

        # Previsão
        y_pred_normalizado = (
            regressor.predict(X_test.reshape(X_test.shape[0], -1))
            if modelo != 'LSTM'
            else regressor.predict(X_test)
        )

        # Criar um novo scaler apenas para 'S'
        scaler_y = MinMaxScaler(feature_range=(0, 1))
        scaler_y.fit(df[['S']])  # Ajusta apenas para a variável alvo

        # Desnormalizar os valores preditos e reais
        y_pred = scaler_y.inverse_transform(y_pred_normalizado.reshape(-1, 1)).flatten()
        y_test_inverso = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()


        # Cálculo das métricas
        mse = mean_squared_error(y_test, y_pred_normalizado)
        rmse = sqrt(mse)
        rmspe = np.sqrt(np.mean(((y_test - y_pred_normalizado) / y_test) ** 2)) * 100
        mape = np.mean(np.abs((y_test - y_pred_normalizado) / y_test)) * 100
        mae = mean_absolute_error(y_test, y_pred_normalizado)
        r2 = r2_score(y_test, y_pred_normalizado)

        # Plotar resultados
        df_previsoes = df.index[split + janela:]
        plotar_resultados(df_previsoes, y_test_inverso, y_pred, trafo, modelo)

        resultados.append({
            'Trafo': trafo,
            'Modelo': modelo,
            'RMSE': np.round(rmse, 4),
            'RMSPE': np.round(rmspe, 4),
            'MAPE': np.round(mape, 4),
            'MAE': np.round(mae, 4),
            'R2': np.round(r2, 4)
        })

    return pd.DataFrame(resultados)


#### Parâmetros pra aprendizado

In [ ]:
mensal = 30
anual = 365

univariado = ['S']

S_GD = ['S', 'PotenciaAcumulada']
S_Grupo = ['S', 'fornec_ALTA', 'fornec_BAIXA']
S_Classe = ['S', 'classe_1', 'classe_2', 'classe_3', 'classe_4', 'classe_5', 'classe_6', 'classe_7', 'classe_8',]

S_GD_Grupo = ['S', 'PotenciaAcumulada', 'fornec_ALTA', 'fornec_BAIXA']
S_GD_Classe = ['S', 'PotenciaAcumulada', 'classe_1', 'classe_2', 'classe_3', 'classe_4', 'classe_5', 'classe_6', 'classe_7', 'classe_8',]
S_Grupo_Classe = ['S', 'fornec_ALTA', 'fornec_BAIXA', 'classe_1', 'classe_2', 'classe_3', 'classe_4', 'classe_5', 'classe_6', 'classe_7', 'classe_8',]

S_GD_Grupo_Classe = ['S', 'PotenciaAcumulada', 'fornec_ALTA', 'fornec_BAIXA', 'classe_1', 'classe_2', 'classe_3', 'classe_4', 'classe_5', 'classe_6', 'classe_7', 'classe_8',]

#### Univariado

In [ ]:
resultados_svr_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='SVR', janela=mensal, features=univariado)
resultados_svr_S.to_csv('SVR_S.csv', sep=';')

In [ ]:
resultados_rfr_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='RFR', janela=mensal, features=univariado)
resultados_rfr_S.to_csv('RFR_S.csv', sep=';')

In [ ]:
resultados_gbr_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='GBR', janela=mensal, features=univariado)
resultados_gbr_S.to_csv('GBR_S.csv', sep=';')

In [ ]:
resultados_lgbm_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LGBM', janela=mensal, features=univariado)
resultados_lgbm_S.to_csv('LGBM_S.csv', sep=';')

In [ ]:
resultados_xgb_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='XGB', janela=mensal, features=univariado)
resultados_xgb_S.to_csv('XGB_S.csv', sep=';')

In [ ]:
resultados_catboost_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='CatBoost', janela=mensal, features=univariado)
resultados_catboost_S.to_csv('CATBOOST_S.csv', sep=';')

In [ ]:
resultados_lstm_S = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LSTM', janela=mensal, features=univariado)
resultados_lstm_S.to_csv('LSTM_S.csv', sep=';')

#### S + GD

In [ ]:
resultados_svr_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='SVR', janela=mensal, features=S_GD)
resultados_svr_S_GD.to_csv('SVR_S_GD.csv', sep=';')

In [ ]:
resultados_rfr_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='RFR', janela=mensal, features=S_GD)
resultados_rfr_S_GD.to_csv('RFR_S_GD.csv', sep=';')

In [ ]:
resultados_gbr_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='GBR', janela=mensal, features=S_GD)
resultados_gbr_S_GD.to_csv('GBR_S_GD.csv', sep=';')

In [ ]:
resultados_lgbm_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LGBM', janela=mensal, features=S_GD)
resultados_lgbm_S_GD.to_csv('LGBM_S_GD.csv', sep=';')

In [ ]:
resultados_xgb_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='XGB', janela=mensal, features=S_GD)
resultados_xgb_S_GD.to_csv('XGB_S_GD.csv', sep=';')

In [ ]:
resultados_catboost_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='CatBoost', janela=mensal, features=S_GD)
resultados_catboost_S_GD.to_csv('CATBOOST_S_GD.csv', sep=';')

In [ ]:
resultados_lstm_S_GD = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LSTM', janela=mensal, features=S_GD)
resultados_lstm_S_GD.to_csv('LSTM_S_GD.csv', sep=';')

#### S + Grupo

In [ ]:
resultados_svr_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='SVR', janela=mensal, features=S_Grupo)
resultados_svr_S_Grupo.to_csv('SVR_S_Grupo.csv', sep=';')

In [ ]:
resultados_rfr_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='RFR', janela=mensal, features=S_Grupo)
resultados_rfr_S_Grupo.to_csv('RFR_S_Grupo.csv', sep=';')

In [ ]:
resultados_gbr_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='GBR', janela=mensal, features=S_Grupo)
resultados_gbr_S_Grupo.to_csv('GBR_S_Grupo.csv', sep=';')

In [ ]:
resultados_lgbm_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LGBM', janela=mensal, features=S_Grupo)
resultados_lgbm_S_Grupo.to_csv('LGBM_S_Grupo.csv', sep=';')

In [ ]:
resultados_xgb_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='XGB', janela=mensal, features=S_Grupo)
resultados_xgb_S_Grupo.to_csv('XGB_S_Grupo.csv', sep=';')

In [ ]:
resultados_catboost_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='CatBoost', janela=mensal, features=S_Grupo)
resultados_catboost_S_Grupo.to_csv('CATBOOST_S_Grupo.csv', sep=';')

In [ ]:
resultados_lstm_S_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LSTM', janela=mensal, features=S_Grupo)
resultados_lstm_S_Grupo.to_csv('LSTM_S_Grupo.csv', sep=';')

#### S + Classe

In [ ]:
resultados_svr_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='SVR', janela=mensal, features=S_Classe)
resultados_svr_S_Classe.to_csv('SVR_S_Classe.csv', sep=';')

In [ ]:
resultados_rfr_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='RFR', janela=mensal, features=S_Classe)
resultados_rfr_S_Classe.to_csv('RFR_S_Classe.csv', sep=';')

In [ ]:
resultados_gbr_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='GBR', janela=mensal, features=S_Classe)
resultados_gbr_S_Classe.to_csv('GBR_S_Classe.csv', sep=';')

In [ ]:
resultados_lgbm_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LGBM', janela=mensal, features=S_Classe)
resultados_lgbm_S_Classe.to_csv('LGBM_S_Classe.csv', sep=';')

In [ ]:
resultados_xgb_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='XGB', janela=mensal, features=S_Classe)
resultados_xgb_S_Classe.to_csv('XGB_S_Classe.csv', sep=';')

In [ ]:
resultados_catboost_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='CatBoost', janela=mensal, features=S_Classe)
resultados_catboost_S_Classe.to_csv('CATBOOST_S_Classe.csv', sep=';')

In [ ]:
resultados_lstm_S_Classe = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LSTM', janela=mensal, features=S_Classe)
resultados_lstm_S_Classe.to_csv('LSTM_S_Classe.csv', sep=';')

#### S + GD + Grupo

In [ ]:
resultados_svr_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='SVR', janela=mensal, features=S_GD_Grupo)
resultados_svr_S_GD_Grupo.to_csv('SVR_S_GD_Grupo.csv', sep=';')

resultados_rfr_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='RFR', janela=mensal, features=S_GD_Grupo)
resultados_rfr_S_GD_Grupo.to_csv('RFR_S_GD_Grupo.csv', sep=';')

resultados_gbr_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='GBR', janela=mensal, features=S_GD_Grupo)
resultados_gbr_S_GD_Grupo.to_csv('GBR_S_GD_Grupo.csv', sep=';')

resultados_lgbm_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LGBM', janela=mensal, features=S_GD_Grupo)
resultados_lgbm_S_GD_Grupo.to_csv('LGBM_S_GD_Grupo.csv', sep=';')

resultados_xgb_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='XGB', janela=mensal, features=S_GD_Grupo)
resultados_xgb_S_GD_Grupo.to_csv('XGB_S_GD_Grupo.csv', sep=';')

resultados_catboost_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='CatBoost', janela=mensal, features=S_GD_Grupo)
resultados_catboost_S_GD_Grupo.to_csv('CATBOOST_S_GD_Grupo.csv', sep=';')

resultados_lstm_S_GD_Grupo = treinar_e_prever_modelo(data, trafos_escolhidos, modelo='LSTM', janela=mensal, features=S_GD_Grupo)
resultados_lstm_S_GD_Grupo.to_csv('LSTM_S_GD_Grupo.csv', sep=';')